In [1]:
import pandas as pd
import glob

In [2]:
path = '/Users/ethanenkhtur/Documents/Codecademy/bike-rental-starter-kit/data/JC*'

csv_files = glob.glob(path)

dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    
df = pd.concat(dataframes, ignore_index=True)

In [7]:
df.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

In [11]:
print(df[df['User Type'].isna()])

        Trip Duration           Start Time            Stop Time  \
8546              943  2016-08-01 09:48:21  2016-08-01 10:04:05   
8634             2319  2016-08-01 12:29:23  2016-08-01 13:08:03   
10848             365  2016-08-03 10:46:37  2016-08-03 10:52:43   
13241             396  2016-08-05 10:20:57  2016-08-05 10:27:34   
13385             581  2016-08-05 14:06:10  2016-08-05 14:15:51   
...               ...                  ...                  ...   
234992            141  2016-05-13 09:25:49  2016-05-13 09:28:11   
235566            126  2016-05-14 12:13:39  2016-05-14 12:15:45   
235577            142  2016-05-14 12:22:41  2016-05-14 12:25:04   
237296            143  2016-05-17 08:19:54  2016-05-17 08:22:17   
239560            164  2016-05-20 09:04:14  2016-05-20 09:06:58   

        Start Station ID Start Station Name  Start Station Latitude  \
8546                3214   Essex Light Rail               40.712774   
8634                3199       Newport Pkwy          

In [12]:
df.count()

Trip Duration              247584
Start Time                 247584
Stop Time                  247584
Start Station ID           247584
Start Station Name         247584
Start Station Latitude     247584
Start Station Longitude    247584
End Station ID             247584
End Station Name           247584
End Station Latitude       247584
End Station Longitude      247584
Bike ID                    247584
User Type                  247204
Birth Year                 228585
Gender                     247584
dtype: int64

In [4]:
weather_file = '/Users/ethanenkhtur/Documents/Codecademy/bike-rental-starter-kit/data/newark_airport_2016.csv'

weather_df = pd.read_csv(weather_file)

weather_df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1
